<a href="https://colab.research.google.com/github/chrober24/SL1_2021/blob/main/stylegan2_tutorial_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stylegan2: Generating images
This will install all the necessary libraries to use the StyleGAN2 repo. Press the play button or `shift+return` to run each cell.

Only run the next cell once per session.


#Setup

This will install all the necessary libraries to use the StyleGAN2 repo. Press the play button or `shift+return` to run each cell.

Only run the next cell once per session. (?unless mounting gdrive?)

In [ ]:
%tensorflow_version 1.x

# Derrick's repo (even if you trained with skyflynil this will work!)
!git clone https://github.com/dvschultz/stylegan2 #use this fork to get interpolation functions
!pip install opensimplex # needed for noise interpolation
%cd stylegan2

TensorFlow 1.x selected.
Cloning into 'stylegan2'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 22.47 MiB | 35.08 MiB/s, done.
Resolving deltas: 100% (285/285), done.
/content/stylegan2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download your trained model from Google Drive
Input the google id of your trained .pkl file. 
The -O option here is to specify the output file name and location.

In [ ]:
# Replace this id with the google id of YOUR trained .pkl file. The id here is for FFHQ.

!gdown --id 1-TCBFYBYVZXgblVH5gitS0m296_MRKVe -O /content/network.pkl

Downloading...
From: https://drive.google.com/uc?id=1-TCBFYBYVZXgblVH5gitS0m296_MRKVe
To: /content/network.pkl
364MB [00:02, 127MB/s] 


# Seed Generation

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results. Truncation is looked at more closely in the next section.

In [ ]:
!python run_generator.py generate-images --network=/content/network.pkl --seeds=1-400 --truncation-psi=0.7

Local submit - run_dir: results/00002-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/network.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 1 (1/400) ...
Generating image for seed 2 (2/400) ...
Generating image for seed 3 (3/400) ...
Generating image for seed 4 (4/400) ...
Generating image for seed 5 (5/400) ...
Generating image for seed 6 (6/400) ...
Generating image for seed 7 (7/400) ...
Generating image for seed 8 (8/400) ...
Generating image for seed 9 (9/400) ...
Generating image for seed 10 (10/400) ...
Generating image for seed 11 (11/400) ...
Generating image for seed 12 (12/400) ...
Generating image for seed 13 (13/400) ...
Generating image for seed 14 (14/400) ...
Generating image for seed 15 (15/400) ...
Generating image for seed 16 (16/400) ...
Generating image

zip the generated files and download them.

In [ ]:
!zip -r generated-t5.zip /content/stylegan2/results/00002-generate-images

updating: content/stylegan2/results/00002-generate-images/ (stored 0%)
updating: content/stylegan2/results/00002-generate-images/seed0268.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0306.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0069.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0344.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0175.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0364.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0143.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0231.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0141.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0262.png (deflated 0%)
updating: content/stylegan2/results/00002-generate-images/seed0218.png (deflate

##Near neighbors
**Added by popular demand**

Let’s say you have a seed you like, but want to see other images like it to see if there’s something better. Now you can with the `near-neighbor` argument.

### Options
`--network`, `--seeds`, and `--truncation_psi` work the same as above.

`--diameter`: this sets how far away from the seed you want to generate images. `.0000001` is really close, `.5` is reallly far.

`--num_samples`: how many samples you want to produce

`--save_vector`: this will save the vector as a file in the .npy format. You can then use this for interpolation (not super well supported right now, but can be used manually—see the Projection code for an example of reading a .npy file and interpolating it).




In [ ]:
!python run_generator.py generate-neighbors --network=/content/network.pkl --seeds=231 --truncation-psi=0.5 --num_samples=4 --diameter=.5 --save_vector

Local submit - run_dir: results/00001-generate-neighbors
dnnlib: Running run_generator.generate_neighbors() on localhost...
Loading networks from "/content/network.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 231 (1/1) ...
dnnlib: Finished run_generator.generate_neighbors() in 16s.


## **Linear Interpolation**

Linear interpolation generate a linear path from one seed to another. The makers of StyleGAN say that doing this in the w space produces the best disentangled interpolations. But let’s start by looking at it in z space.

`--seeds`: Use images you generated to control the interpolation points. If your first and last seed are the same this will produce a loop (nice for Instagram and gifs!)

In [ ]:
!python run_generator.py generate-latent-walk --network=/content/network.pkl --walk-type='line-z' --seeds=1570,1508 --frames 240 --truncation-psi=0.7

produce a video from the frames


In [ ]:
!ffmpeg -r 24 -i ./results/00003-generate-latent-walk/frame%05d.png -vcodec libx264 -pix_fmt yuv420p interpolation_6_halfspeed.mp4

# Truncation
Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically its infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 
`--network`: Again, point this to your .pkl file.

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

In [ ]:
!python run_generator.py truncation-traversal --network=/content/network.pkl --seed=18 --start=-1.0 --stop=1.0 --increment=0.03

In [ ]:
listOfImageNames = ['/content/stylegan2/results/00002-truncation-traversal/frame00001.png',
                    '/content/stylegan2/results/00002-truncation-traversal/frame00011.png',
                    '/content/stylegan2/results/00002-truncation-traversal/frame00021.png',
                    '/content/stylegan2/results/00002-truncation-traversal/frame00031.png',
                    '/content/stylegan2/results/00002-truncation-traversal/frame00040.png',]

for imageName in listOfImageNames:
    display(Image(filename=imageName, width=400))

make these frames into a video using ffmpeg. Make sure the path to your results folder is correct and give the mp4 an unique name.


In [ ]:
!ffmpeg -r 24 -i /content/stylegan2/results/00002-truncation-traversal/frame%05d.png -vcodec libx264 -pix_fmt yuv420p truncation-example.mp4

#Projection
Projection is the process of taking an image from outside the model and finding it’s nearest representation inside the model.

For projection, we’re gonna do something a little different in this notebook and load the code directly into Colab. Don’t worry if this looks scary—there’s only two things you want to edit in the code for any projection.

download this pretrained pkl file. This allows us to calculate how different our projection image is from the image in the model.

In [ ]:
!mkdir pkl
!gdown --id 1gbxwfHNOaGjGsLNTmmSrNA85X2VWHHOq -O /content/stylegan2/pkl/vgg16_zhang_perceptual.pkl

Next, all the code we need.

In [ ]:
network_pkl = "/content/network.pkl"

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio

import pretrained_networks

# Convert uploaded images to TFRecords
import dataset_tool

# Run the projector
import training.dataset
import training.misc
import projector
import os 

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def interpolate(zs, steps,type='linear'):
  out = []
  for i in range(len(zs)-1):
    c = zs[i+1]-zs[i]

    for index in range(steps):
      fraction = index/float(steps) # t/d
      
      # translated from: https://github.com/danro/jquery-easing/blob/master/jquery.easing.js
      # see https://easings.net/ for examples
      if type == 'linear':
        out.append( c * fraction + zs[i] ) # c*(t/d)+b
      elif type == 'easeInSine':
        out.append( -c * np.cos(fraction * (np.pi/2)) + c + zs[i] ) # -c * Math.cos(t/d * (Math.PI/2)) + c + b
      elif type == 'easeOutSine':
        out.append( c * np.sin(fraction * (np.pi/2)) + zs[i]) # c * Math.sin(t/d * (Math.PI/2)) + b
      elif type == 'easeInOutSine':
        out.append(-c/2 * (np.cos(np.pi*fraction) - 1.0) + zs[i]) # -c/2 * (Math.cos(Math.PI*t/d) - 1) + b;
      elif type == 'easeInQuad':
        out.append(c * fraction * fraction + zs[i]) # c*(t/=d)*t + b;
      elif type == 'easeOutQuad':
       out.append(-c * fraction * (fraction-2) + zs[i]) # -c *(t/=d)*(t-2) + b;
      # elif type == 'easeInOutQuad':
      #   if(fraction/2 < 1):
      #     out.append( ((c/2)*fraction*fraction) + zs[i]) #if ((t/=d/2) < 1) return c/2*t*t + b;
      #   else:
		  #     out.append( (-c/2) * ((index-=1)*(index-2) - 1) + zs[i]; #return -c/2 * ((--t)*(t-2) - 1) + b;
      else: 
        out.append( c * fraction + zs[i] ) # c*(t/d)+b
  return out

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location + ('%05d.png' % (idx))
    img.save(file)

def generate_images_in_w_space(dlatents, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    imgs = []
    for row, dlatent in enumerate(dlatents):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs  

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

Next, we need to make some folders so we can upload the image for projection. Only run this cell once per session.

In [ ]:
!mkdir projection
!mkdir projection/imgs
!mkdir projection/out

# Now upload a single image to 'stylegan2/projection/imgs' (use the Files side panel). Image should be color PNG, with a size of 1024x1024.

In [ ]:
!rm projection/records/*.*
!rm projection/out/*.*

In [ ]:
!rm /content/stylegan2/projection/out/*.*

Now upload a single image to stylegan2/projection/imgs (use the Files side panel). Image should be color PNG, with a size of 1024x1024.

Things you might want to edit in here:

`file_name`: I recommend saving this to be something readable in case you want to do interpolations with it. Change it every time you switch projection samples.

`proj.num_steps`: this is the number of iterations you want the model to run thru. Higher numbers will increase accuracy but also take longer to produce.

In [ ]:
# Convert uploaded images to TFRecords
dataset_tool.create_from_images("./projection/records/", "./projection/imgs/", True)

!rm projection/out/*.*

def project_image_run(proj, targets, png_prefix, num_snapshots):
    snapshot_steps = set(proj.num_steps - np.linspace(0, proj.num_steps, num_snapshots, endpoint=False, dtype=int))
    training.misc.save_image_grid(targets, png_prefix + 'target.png', drange=[-1,1])
    proj.start(targets)
    while proj.get_cur_step() < proj.num_steps:
        print('\r%d / %d ... ' % (proj.get_cur_step(), proj.num_steps), end='', flush=True)
        proj.step()
        if proj.get_cur_step() in snapshot_steps:
            training.misc.save_image_grid(proj.get_images(), png_prefix + 'step%04d.png' % proj.get_cur_step(), drange=[-1,1])
    print('\r%-30s\r' % '', end='', flush=True)
    type(proj.get_noises())
    #if you want to do interpolations, name the file below something memorable
    file_name = 'mug'
    np.save(T('./projection/'+file_name+'.npy'), proj.get_dlatents())

def project_real_images(dataset_name, data_dir, num_images, num_snapshots):
    proj = projector.Projector()
    proj.set_network(Gs)
    #num_steps = how many iterations; larger = ~more accurate but longer run times 
    proj.num_steps=300

    print('Loading images from "%s"...' % dataset_name)
    dataset_obj = training.dataset.load_dataset(data_dir=data_dir, tfrecord_dir=dataset_name, max_label_size=0, verbose=True, repeat=False, shuffle_mb=0)
    print(dataset_obj.shape, Gs.output_shape[1:])
    assert dataset_obj.shape == Gs.output_shape[1:]

    for image_idx in range(num_images):
        print('Projecting image %d/%d ...' % (image_idx, num_images))
        images, _labels = dataset_obj.get_minibatch_np(1)
        images = training.misc.adjust_dynamic_range(images, [0, 255], [-1, 1])
        project_image_run(proj, targets=images, png_prefix=dnnlib.make_run_dir_path('projection/out/image%04d-' % image_idx), num_snapshots=num_snapshots)

project_real_images("records","./projection",1,100)

## Projection process video
Want to make one of those really cool projection videos? Run the code below.

In [ ]:
# Create video 
import glob

imgs = sorted(glob.glob("projection/out/*step*.png"))

target_imgs = sorted(glob.glob("projection/out/*target*.png"))
assert len(target_imgs) == 1, "More than one target found?"
target_img = imageio.imread(target_imgs[0])

movieName = "projection/movie.mp4"
with imageio.get_writer(movieName, mode='I') as writer:
    for filename in imgs:
        image = imageio.imread(filename)

        # Concatenate images with original target image
        w,h = image.shape[0:2]
        canvas = PIL.Image.new('RGBA', (w*2,h), 'white')
        canvas.paste(Image.fromarray(target_img), (0, 0))
        canvas.paste(Image.fromarray(image), (w, 0))

        writer.append_data(np.array(canvas))

### Make an interpolation!

Go back to the cell where you generate the projection and run it again. Make sure you use a different file name for `.npy` file.

If you generate a couple different projections, you can then interpolate between the different w values. 

In [ ]:
latent1 = np.load('/content/bladerunner.npy')
latent2 = np.load('/content/stylegan2/projection/c3po.npy')
frame_count = 144

imgs = generate_images_in_w_space(interpolate([latent1,latent2],frame_count,'linear'),0.7)

!rm -rf interpolations
%mkdir interpolations
saveImgs(imgs,'./interpolations/')

turn that interpolation into a video.

In [ ]:
!ffmpeg -r 24 -i ./interpolations/%05d.png -vcodec libx264 -pix_fmt yuv420p deckard-c3p0-v5.mp4